In [ ]:
import numpy as np
import cv2
import os
import re
from scipy import ndimage
from tqdm import tqdm
import keras
from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, Reshape, UpSampling2D, Flatten, Dense, Lambda, BatchNormalization
from keras.metrics import BinaryAccuracy
from keras.models import Model
import keras.backend as K
import tensorflow as tf
from keras.losses import mse

from google.colab.patches import cv2_imshow
from google.colab import drive

import shutil

In [ ]:
!mkdir /content/images
!mkdir /content/images2

drive.mount('/content/drive')

!unzip -q '/content/drive/My Drive/NATO/chooseFrom.zip' -d '/content/images'
!unzip -q '/content/drive/My Drive/NATO/test_images.zip' -d '/content/images2'

In [ ]:
Dict = {0: 'advance_to_contact', 2: 'attack', 9: 'counterattack', 18: 'main_attack'}
# Dict = {0: 'otse_k', 1: 'otse_l', 2: 'parem_n', 3: 'parem_y', 4: 'vasak_n', 5: 'vasak_y'}
Dict_images = {key: [] for key in Dict.values()}

In [ ]:
def cut_excess_white(symbol, excess_str = 120):
    symbol = symbol[np.argwhere(np.amin(symbol,axis=1) < excess_str)[0][0]:np.argwhere(np.amin(symbol,axis=1) < excess_str)[-1][0],:]
    symbol = symbol[:,np.argwhere(np.amin(symbol,axis=0) < excess_str)[0][0]:np.argwhere(np.amin(symbol,axis=0) < excess_str)[-1][0]]
    return symbol

In [ ]:
# data = '/content/images2'
# file_list=os.listdir(data)

# symbols_regex = '([a-zA-Z_ ]*)\d*.*'
# for filename in tqdm(file_list):
#    img = cv2.imread(data+"/"+filename, 0)
#    img[img <= 100] = 0
#    img[img > 100] = 255
#    img = cut_excess_white(img)
#    key = re.findall(symbols_regex, filename)[0]
#    Dict_images[key].append(img)

In [ ]:
data = '/content/images'
file_list=os.listdir(data)

dir_regex = '([a-zA-Z_ ]*)\d*.*'
symbols_regex = '([a-zA-Z_ ]*)\d*.*'
for dir in file_list:
    for filename in tqdm(os.listdir(data+"/"+dir)):
        img = cv2.imread(data+"/"+dir+"/"+filename, 0)
        img[img <= 100] = 0
        img[img > 100] = 255
        img = cut_excess_white(img)
        key = re.findall(symbols_regex, dir)[0]
        key2 = re.findall(symbols_regex, filename)[0]
        Dict_images[key].append([img, str(key2)])

In [ ]:
Dict_images_all = {key: [] for key in Dict.values()}

In [ ]:
def rotate_images(image, symbol):
    for degree in range(0, 1, 5):
        rotation_angle = degree
        img = ndimage.rotate(image, rotation_angle, mode='constant', cval=255)
        dim = (80, 80)
        resized_img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        binImg = np.where(resized_img <= 150, 0, 255)
        Dict_images_all[symbol].append(binImg)

In [ ]:
for i in Dict_images.keys():
    for img in tqdm(Dict_images[str(i)]):
        rotate_images(img[0], str(i))

In [ ]:
def Sampling(inputs):
    mean, log_var = inputs
    return K.random_normal(tf.shape(mean)) * K.exp(log_var / 2) + mean

def model_version12():
    # autoencoder model
    beta_value = 50
    # encoder
    input = Input(shape=(80, 80, 1))
    x = Conv2D(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(input)
    x = Conv2D(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(x)
    x = Conv2D(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(x)
    x = Conv2D(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    codings_mean = keras.layers.Dense(10, name="encoder_mean")(x)
    codings_log_var = keras.layers.Dense(10, name="encoder_log_var")(x)
    codings = Lambda(Sampling, name="encoder_output")([codings_mean, codings_log_var])
    variational_encoder = keras.models.Model(inputs=[input], outputs=[codings_mean, codings_log_var, codings], name="encoder")

    #decoder
    decoder_inputs = keras.layers.Input(shape=(10))
    z = Dense(256, activation='relu')(decoder_inputs)
    z = Dense(256, activation='relu')(z)
    z = Dense(5*5*32, activation='relu')(z)
    z = Reshape((5, 5, 32))(z)
    z = Conv2DTranspose(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(z)
    z = Conv2DTranspose(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(z)
    z = Conv2DTranspose(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(z)
    z = Conv2DTranspose(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(z)
    output = Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(z)
    variational_decoder = keras.models.Model(inputs=[decoder_inputs], outputs=[output], name="decoder")

    _, _, codings = variational_encoder(input)
    reconstructions = variational_decoder(codings)
    variational_ae = keras.models.Model(inputs=[input], outputs=[reconstructions], name="autoencoder")

    reconstruction_loss_factor = 1000
    reconstruction_loss = mse(K.flatten(input), K.flatten(reconstructions))
    reconstruction_loss *= 80 * 80 * 1
    kl_loss = -0.5 * beta_value * K.sum(1 + codings_log_var - K.square(codings_mean) - K.exp(codings_log_var), axis=1)
    vae_loss = K.mean(reconstruction_loss_factor * reconstruction_loss + kl_loss)
    variational_ae.add_loss(vae_loss)

    variational_ae.add_metric(kl_loss, name="kl_loss")
    variational_ae.add_metric(reconstruction_loss, name="reconstruction_loss")

    return variational_ae

In [ ]:
model = model_version12()

for i in Dict_images_all.keys():
    print(str(i))
    model.load_weights('/content/drive/MyDrive/models_trajectory_final/'+str(i)+'_autoencoder_model12_b50_10.h5')
    encoder = Model(inputs=model.input, outputs=model.get_layer("encoder").output)

    extra_labels = []
    for img in Dict_images[str(i)]:
        extra_labels.append(img[1])


    images = np.array(Dict_images_all[str(i)])
    images = images.astype('float32') / 255.
    images = images.reshape(images.shape[0], images.shape[1], images.shape[2], 1)

    _, _, features = encoder.predict(images)
    data = {"images": Dict_images_all[str(i)], "features": features, "extra_label": extra_labels}
    np.save(str(i)+"_data_model12_b50_10.npy", data)

    colab_link = '/content/'+str(i)+'_data_model12_b50_10.npy'
    gdrive_link = "/content/drive/My Drive/npy_final/"
    shutil.copy(colab_link, gdrive_link)